In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import warnings; warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
import mglearn
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (confusion_matrix,precision_score,recall_score,f1_score,
    roc_curve,roc_auc_score,precision_recall_curve,accuracy_score,classification_report)

In [2]:
data = pd.read_csv("Energi_Viborg_Dandas_data.csv")

#drop columns not needed after asking the company about the meaning of these features

columns_to_be_removed = ['ID', 'mslink', 'XKoordinat','YKoordinat','LedningID','Dobbeltled','EjerKompon','SystemKode','KategoriAf','DatoUdf']
data=data.drop(columns_to_be_removed,axis='columns')

# in the column DatoSaneri is the date of repairing and if there is no date it means it is not repaired

data['DatoSaneri'].fillna(0, inplace=True)

# take only the pipes that are broken(by TV insection) now and the repaired ones

data_with_TVObsAndSaneri = data[data['TVObsKode'].isin([1]) | data['DatoSaneri'] > 0]

In [3]:
#get around 2077 rows with not broken pipes

data_not_broken = data[~data['TVObsKode'].isin([0]) | data['DatoSaneri'] == 0]
data_not_broken = data_not_broken.sample(n=2077) 

In [4]:
frames = [data_with_TVObsAndSaneri, data_not_broken]
  
data_final = pd.concat(frames)
data_final

,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri
36,34.72,33.480000,64.88,19.112207,300.0,1.0,1939.0,1,0,0.0,2010,2014,1997.0
42,39.46,39.160000,91.75,3.269755,400.0,1.0,1939.0,1,0,1.0,2010,2014,0.0
43,39.71,39.480000,87.69,2.622876,300.0,1.0,1939.0,1,0,1.0,2010,2014,0.0
64,40.55,40.080000,52.11,9.019382,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0
65,40.38,40.550000,68.39,-2.485744,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2142,26.85,25.740000,30.10,36.877076,600.0,1.0,1941.0,1,0,0.0,2010,2014,0.0
6450,40.28,38.140000,29.76,71.908602,200.0,1.0,1964.0,1,0,0.0,2010,2014,0.0
11967,20.77,20.440000,58.53,5.638134,700.0,1.0,1971.0,1,0,0.0,2010,2014,0.0
8492,41.53,41.380000,20.96,7.156489,250.0,1.0,1995.0,1,0,0.0,2010,2014,0.0


In [5]:
data = data_final

In [6]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
print("Number of rows before removing NaNs: {}".format(data.shape[0]))
data = data.dropna()
print("Number of rows after removing NaNs: {}".format(data.shape[0]))

Number of rows before removing NaNs: 4154
Number of rows after removing NaNs: 4154


In [7]:
#get data copied
datacopy = data


# add  age column

#get current year
from datetime import date
now = date.today().year


def age_df(datacopy):

    if (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] > 0) :
        return (now - datacopy['DatoSaneri'])
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri']== 0):
        return (now - datacopy['anlag_aar'])
    elif (datacopy['TVObsKode'] == 0) and (datacopy['DatoSaneri'] > 0):
        return (now - datacopy['DatoSaneri'])
    elif (datacopy['TVObsKode']== 0) and (datacopy['DatoSaneri']== 0):
        return (now - datacopy['anlag_aar'])

datacopy['Age'] = datacopy.apply(age_df, axis = 1)

In [8]:
# add a column 'PipeStatus'
# 1 as broken and 0 as not broken

def broken_df(datacopy):

    if (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] < (datacopy['DatoOpdate'])) and (datacopy['DatoSaneri'] != 0):
        return 1
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] >= (datacopy['DatoOpdate'])) and (datacopy['DatoSaneri'] != 0):
        return 0
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri']== 0):
        return 1
    elif (datacopy['TVObsKode'] == 0) and (datacopy['DatoSaneri'] > 0):
        return 0
    elif (datacopy['TVObsKode']== 0) and (datacopy['DatoSaneri']== 0):
        return 0

datacopy['PipeStatus'] = datacopy.apply(broken_df, axis = 1)

In [9]:
# remove NaNs
print("Number of rows before removing NaNs: {}".format(datacopy.shape[0]))
datacopy = datacopy.dropna()
print("Number of rows after removing NaNs: {}".format(datacopy.shape[0]))

Number of rows before removing NaNs: 4154
Number of rows after removing NaNs: 4154


In [10]:
#drop columns not needed after adding new features

columns_to_be_removed = ['DatoOprett', 'DatoOpdate']
datacopy=datacopy.drop(columns_to_be_removed,axis='columns')
datacopy[0:-1]

,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoSaneri,Age,PipeStatus
36,34.72,33.48,64.88,19.112207,300.0,1.0,1939.0,1,0,0.0,1997.0,24.0,0
42,39.46,39.16,91.75,3.269755,400.0,1.0,1939.0,1,0,1.0,0.0,82.0,1
43,39.71,39.48,87.69,2.622876,300.0,1.0,1939.0,1,0,1.0,0.0,82.0,1
64,40.55,40.08,52.11,9.019382,250.0,1.0,1945.0,1,0,1.0,0.0,76.0,1
65,40.38,40.55,68.39,-2.485744,250.0,1.0,1945.0,1,0,1.0,0.0,76.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14127,42.73,41.00,29.47,58.703767,200.0,1.0,1964.0,1,0,0.0,0.0,57.0,0
2142,26.85,25.74,30.10,36.877076,600.0,1.0,1941.0,1,0,0.0,0.0,80.0,0
6450,40.28,38.14,29.76,71.908602,200.0,1.0,1964.0,1,0,0.0,0.0,57.0,0
11967,20.77,20.44,58.53,5.638134,700.0,1.0,1971.0,1,0,0.0,0.0,50.0,0


In [11]:
# creating features set and target

columns_to_be_removed = ['PipeStatus']
data_features= datacopy.drop(columns_to_be_removed,axis='columns')
columns_to_be_removed = ['fra_kote','til_kote', 'Laengde','Fald','DiameterIn','MaterialeK','anlag_aar','TransportK','Funktionsk','TVObsKode','DatoSaneri','Age']
data_target=datacopy.drop(columns_to_be_removed,axis='columns')

In [12]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
print("Number of rows before removing NaNs: {}".format(data.shape[0]))
data = data.dropna()
print("Number of rows after removing NaNs: {}".format(data.shape[0]))

Number of rows before removing NaNs: 4154
Number of rows after removing NaNs: 4154


# Tuning hyperparameters with train-test split and grid search cross validation

In [13]:
# Divide the data into training and test
X_train, X_test, y_train, y_test = train_test_split(
    data_features, data_target, stratify=data_target, random_state=42)

In [25]:
#List Hyperparameters that we want to tune by cross validation

kernel = ['linear', 'rbf']
C = [100, 10, 1.0, 0.1, 0.01, 0.001]
gamma = 10**np.linspace(-5, 5, 10)

#Convert to dictionary
hyperparameters = dict(C = C ,gamma= gamma, kernel= kernel)

#Create new logistic object
svm = SVC()

#Use GridSearch
clf = GridSearchCV(svm, hyperparameters, cv=5,return_train_score=True, scoring="recall")

#Fit the model
best_model = clf.fit(X_train, y_train)

#Print The value of best Hyperparameters
print('Best C:', best_model.best_estimator_.get_params()['C'])
print('Best kernel:', best_model.best_estimator_.get_params()['kernel'])
print('Best gamma:', best_model.best_estimator_.get_params()['gamma'])    
print("Accuracy on training set: {:.3f}".format(best_model.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(best_model.score(X_test, y_test)))

Best C: 10
Best kernel: linear
Best gamma: 1e-05
Accuracy on training set: 0.974
Accuracy on test set: 0.959


In [26]:
#Create SVM Object.
a = best_model.best_estimator_.get_params()['C']
b = best_model.best_estimator_.get_params()['kernel']
c = best_model.best_estimator_.get_params()['gamma']
svm = SVC(C = a, kernel= b, gamma = c)

#Create x and y variables.
x = data_features
y = data_target

#Split data into training and testing.
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=42)

#Training the model.
svm.fit(X_train, y_train)

#Predict test data set.
y_pred =svm.predict(X_test)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       835
           1       0.99      0.98      0.98       204

    accuracy                           0.99      1039
   macro avg       0.99      0.99      0.99      1039
weighted avg       0.99      0.99      0.99      1039



0.985948690853587

# Tuning hyperparameters with validation set split

In [27]:
# Divide the data into training, test and validation

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, random_state=43)
print("Size of training set:{}".format(X_train.shape[0]))
print("Size of validation set:{}".format(X_val.shape[0]))
print("Size of test set:{}".format(X_test.shape[0]))

Size of training set:2336
Size of validation set:779
Size of test set:1039


In [28]:
best_score = 0

for C in [100, 10, 1.0, 0.1, 0.01, 0.001]:
    for kernel in ['linear', 'rbf']:
        for gamma in 10**np.linspace(-5, 5, 10):
        
            # Learn the model
            svm = SVC(kernel=kernel, C=C, gamma = gamma)
            svm.fit(X_train, y_train)
        
            # Evaluate the model
            score = svm.score(X_val, y_val)
        
            # If improvement, store score and parameter
            if score>best_score:
                best_score = score
                best_C = C
                best_kernel= kernel
                best_gamma= gamma        


# Build a model on the combine training and valiation data
svm = SVC(kernel= best_kernel, C =best_C, gamma = best_gamma)
svm.fit(X_trainval, y_trainval)

print("Best best_C found: {}".format(best_C))
print("Best best_kernel found: {}".format(best_kernel))
print("Best best_gamma found: {}".format(best_gamma))
print("Best score on validation set: {}".format(best_score))
print("Score on training/validation set: {}".format(svm.score(X_trainval, y_trainval)))
print("Score on test set: {}".format(svm.score(X_test, y_test)))

Best best_C found: 1.0
Best best_kernel found: linear
Best best_gamma found: 1e-05
Best score on validation set: 0.9897304236200257
Score on training/validation set: 0.9913322632423756
Score on test set: 0.9932627526467758


In [29]:
#Predict test data set.
y_pred = svm.predict(X_test)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       835
           1       0.97      1.00      0.98       204

    accuracy                           0.99      1039
   macro avg       0.99      0.99      0.99      1039
weighted avg       0.99      0.99      0.99      1039



0.9939562052365857

# Tuning parameters with cross validation split

In [30]:
# Divide the data into training and test , no validation data

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)
print("Size of training set:{}".format(X_train.shape[0]))
print("Size of test set:{}".format(X_test.shape[0]))

Size of training set:2336
Size of test set:1039


In [31]:
best_score = 0

for C in [100, 10, 1.0, 0.1, 0.01, 0.001]:
    for kernel in ['linear', 'rbf']:
        for gamma in 10**np.linspace(-5, 5, 10):
            # Learn the model 
            svm = SVC(kernel=kernel, C=C, gamma = gamma)
            
            # Perform cross validation
            scores = cross_val_score(svm, X_trainval, y_trainval, cv=5)
        
            # Compute the mean score
            score = scores.mean()
        
        
            # If improvement, store score and parameter
            if score>best_score:
                best_score = score
                best_C = C
                best_kernel= kernel
                best_gamma= gamma

# Build a model on the combine training and valiation data
svm = SVC(kernel= best_kernel, C=best_C, gamma = best_gamma)
svm.fit(X_trainval, y_trainval)

print("Best best_C found: {}".format(best_C))
print("Best best_kernel found: {}".format(best_kernel))
print("Best best_gamma found: {}".format(best_gamma))
print("Best average score: {}".format(best_score))
print("Score on training/validation set: {}".format(svm.score(X_trainval, y_trainval)))
print("Score on test set: {}".format(svm.score(X_test, y_test)))

Best best_C found: 100
Best best_kernel found: linear
Best best_gamma found: 1e-05
Best average score: 0.9916532905296951
Score on training/validation set: 0.9903691813804173
Score on test set: 0.9942252165543792


In [32]:
#Predict test data set.
y_pred = svm.predict(X_test)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       835
           1       1.00      0.97      0.99       204

    accuracy                           0.99      1039
   macro avg       1.00      0.99      0.99      1039
weighted avg       0.99      0.99      0.99      1039



0.9852941176470589

# Tuning hyperparameters with train-test split and Normalisation

In [33]:
# Divide the data into training and test
X_train, X_test, y_train, y_test = train_test_split(
    data_features, data_target, stratify=data_target, random_state=42)

# preprocessing using 0-1 scaling
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [34]:
#List Hyperparameters that we want to tune by cross validation

kernel = ['linear', 'rbf']
C = [100, 10, 1.0, 0.1, 0.01, 0.001]
gamma = 10**np.linspace(-5, 5, 10)

#Convert to dictionary
hyperparameters = dict(C = C ,gamma= gamma, kernel= kernel)

#Create new logistic object
svm = SVC()

#Use GridSearch
clf = GridSearchCV(svm, hyperparameters, cv=5,return_train_score=True, scoring="recall")

#Fit the model
best_model = clf.fit(X_train_scaled, y_train)

#Print The value of best Hyperparameters
print('Best C:', best_model.best_estimator_.get_params()['C'])
print('Best kernel:', best_model.best_estimator_.get_params()['kernel'])
print('Best gamma:', best_model.best_estimator_.get_params()['gamma'])    
print("Accuracy on training set: {:.3f}".format(best_model.score(X_train_scaled, y_train)))
print("Accuracy on test set: {:.3f}".format(best_model.score(X_test_scaled, y_test)))

Best C: 100
Best kernel: rbf
Best gamma: 0.0001291549665014884
Accuracy on training set: 1.000
Accuracy on test set: 1.000


In [35]:
#Create SVM Object.
a = best_model.best_estimator_.get_params()['C']
b = best_model.best_estimator_.get_params()['kernel']
c = best_model.best_estimator_.get_params()['gamma']

svm = SVC(C = a, kernel= b, gamma = c)

#Training the model.
svm.fit(X_train_scaled, y_train)

#Predict test data set.
y_pred =svm.predict(X_test_scaled)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       818
           1       0.94      1.00      0.97       221

    accuracy                           0.99      1039
   macro avg       0.97      0.99      0.98      1039
weighted avg       0.99      0.99      0.99      1039



0.991442542787286

# Tuning hyperparameters with train-test split and standardization

In [36]:
# Divide the data into training and test
X_train, X_test, y_train, y_test = train_test_split(
    data_features, data_target, stratify=data_target, random_state=42)

# preprocessing using zero mean and unit variance scaling
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [37]:
#List Hyperparameters that we want to tune by cross validation

kernel = ['linear', 'rbf']
C = [100, 10, 1.0, 0.1, 0.01, 0.001]
gamma = 10**np.linspace(-5, 5, 10)

#Convert to dictionary
hyperparameters = dict(C = C ,gamma= gamma, kernel= kernel)

#Create new logistic object
svm = SVC()

#Use GridSearch
clf = GridSearchCV(svm, hyperparameters, cv=5,return_train_score=True, scoring="recall")

#Fit the model
best_model = clf.fit(X_train_scaled, y_train)

#Print The value of best Hyperparameters
print('Best C:', best_model.best_estimator_.get_params()['C'])
print('Best kernel:', best_model.best_estimator_.get_params()['kernel'])
print('Best gamma:', best_model.best_estimator_.get_params()['gamma'])    
print("Accuracy on training set: {:.3f}".format(best_model.score(X_train_scaled, y_train)))
print("Accuracy on test set: {:.3f}".format(best_model.score(X_test_scaled, y_test)))

Best C: 100
Best kernel: rbf
Best gamma: 1e-05
Accuracy on training set: 1.000
Accuracy on test set: 1.000


In [38]:
#Create SVM Object.
a = best_model.best_estimator_.get_params()['C']
b = best_model.best_estimator_.get_params()['kernel']
c = best_model.best_estimator_.get_params()['gamma']

svm = SVC(C = a, kernel= b, gamma = c)

#Training the model.
svm.fit(X_train_scaled, y_train)

#Predict test data set.
y_pred =svm.predict(X_test_scaled)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       818
           1       0.94      1.00      0.97       221

    accuracy                           0.99      1039
   macro avg       0.97      0.99      0.98      1039
weighted avg       0.99      0.99      0.99      1039



0.991442542787286

# Tuning hyperparameters with validation set split and Normalisation

In [39]:
# Divide the data into training, test and validation

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, random_state=43)

# preprocessing using 0-1 scaling
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)
X_trainval_scaled = scaler.transform( X_trainval)

print("Size of training set:{}".format(X_train_scaled.shape[0]))
print("Size of validation set:{}".format(X_val_scaled.shape[0]))
print("Size of test set:{}".format(X_test_scaled.shape[0]))

Size of training set:2336
Size of validation set:779
Size of test set:1039


In [40]:
best_score = 0

for C in [100, 10, 1.0, 0.1, 0.01, 0.001]:
    for kernel in ['linear', 'rbf']:
        for gamma in 10**np.linspace(-5, 5, 10):
        
            # Learn the model
            svm = SVC(kernel=kernel, C=C, gamma = gamma)
            svm.fit(X_train_scaled, y_train)
        
            # Evaluate the model
            score = svm.score(X_val_scaled, y_val)
        
            # If improvement, store score and parameter
            if score>best_score:
                best_score = score
                best_C = C
                best_kernel= kernel
                best_gamma= gamma        


# Build a model on the combine training and valiation data
svm = SVC(kernel= best_kernel, C =best_C, gamma = best_gamma)
svm.fit(X_trainval_scaled, y_trainval)

print("Best best_C found: {}".format(best_C))
print("Best best_kernel found: {}".format(best_kernel))
print("Best best_gamma found: {}".format(best_gamma))
print("Best score on validation set: {}".format(best_score))
print("Score on training/validation set: {}".format(svm.score(X_trainval_scaled, y_trainval)))
print("Score on test set: {}".format(svm.score(X_test_scaled, y_test)))

Best best_C found: 100
Best best_kernel found: rbf
Best best_gamma found: 0.2782559402207126
Best score on validation set: 0.9961489088575096
Score on training/validation set: 0.9971107544141252
Score on test set: 0.9980750721847931


In [41]:
#Predict test data set.
y_pred = svm.predict(X_test_scaled)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       835
           1       1.00      1.00      1.00       204

    accuracy                           1.00      1039
   macro avg       1.00      1.00      1.00      1039
weighted avg       1.00      1.00      1.00      1039



0.9969502172126334

# Tuning hyperparameters with validation set split and standardization

In [42]:
# Divide the data into training, test and validation

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, random_state=43)

# preprocessing using zero mean and unit variance scaling
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)
X_trainval_scaled = scaler.transform( X_trainval)

print("Size of training set:{}".format(X_train_scaled.shape[0]))
print("Size of validation set:{}".format(X_val_scaled.shape[0]))
print("Size of test set:{}".format(X_test_scaled.shape[0]))

Size of training set:2336
Size of validation set:779
Size of test set:1039


In [43]:
best_score = 0

for C in [100, 10, 1.0, 0.1, 0.01, 0.001]:
    for kernel in ['linear', 'rbf']:
        for gamma in 10**np.linspace(-5, 5, 10):
        
            # Learn the model
            svm = SVC(kernel=kernel, C=C, gamma = gamma)
            svm.fit(X_train_scaled, y_train)
        
            # Evaluate the model
            score = svm.score(X_val_scaled, y_val)
        
            # If improvement, store score and parameter
            if score>best_score:
                best_score = score
                best_C = C
                best_kernel= kernel
                best_gamma= gamma        


# Build a model on the combine training and valiation data
svm = SVC(kernel= best_kernel, C =best_C, gamma = best_gamma)
svm.fit(X_trainval_scaled, y_trainval)

print("Best best_C found: {}".format(best_C))
print("Best best_kernel found: {}".format(best_kernel))
print("Best best_gamma found: {}".format(best_gamma))
print("Best score on validation set: {}".format(best_score))
print("Score on training/validation set: {}".format(svm.score(X_trainval_scaled, y_trainval)))
print("Score on test set: {}".format(svm.score(X_test_scaled, y_test)))

Best best_C found: 100
Best best_kernel found: linear
Best best_gamma found: 1e-05
Best score on validation set: 0.9948652118100129
Score on training/validation set: 0.9990369181380417
Score on test set: 0.9980750721847931


In [44]:
#Predict test data set.
y_pred = svm.predict(X_test_scaled)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       835
           1       1.00      1.00      1.00       204

    accuracy                           1.00      1039
   macro avg       1.00      1.00      1.00      1039
weighted avg       1.00      1.00      1.00      1039



0.9969502172126334

# Tuning hyperparameters with cross validation split and Normalisation

In [45]:
# Divide the data into training and test , no validation data

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)

# preprocessing using 0-1 scaling
scaler = MinMaxScaler()
scaler.fit(X_train)

X_test_scaled = scaler.transform(X_test)
X_trainval_scaled = scaler.transform( X_trainval)


print("Size of training set:{}".format(X_trainval_scaled.shape[0]))
print("Size of test set:{}".format(X_test_scaled.shape[0]))

Size of training set:3115
Size of test set:1039


In [46]:
best_score = 0

for C in [100, 10, 1.0, 0.1, 0.01, 0.001]:
    for kernel in ['linear', 'rbf']:
        for gamma in 10**np.linspace(-5, 5, 10):
            # Learn the model 
            svm = SVC(kernel=kernel, C=C, gamma = gamma)
            
            # Perform cross validation
            scores = cross_val_score(svm, X_trainval_scaled, y_trainval, cv=5)
        
            # Compute the mean score
            score = scores.mean()
        
        
            # If improvement, store score and parameter
            if score>best_score:
                best_score = score
                best_C = C
                best_kernel= kernel
                best_gamma= gamma

# Build a model on the combine training and valiation data
svm = SVC(kernel= best_kernel, C=best_C, gamma = best_gamma)
svm.fit(X_trainval_scaled, y_trainval)

print("Best best_C found: {}".format(best_C))
print("Best best_kernel found: {}".format(best_kernel))
print("Best best_gamma found: {}".format(best_gamma))
print("Best average score: {}".format(best_score))
print("Score on training/validation set: {}".format(svm.score(X_trainval_scaled, y_trainval)))
print("Score on test set: {}".format(svm.score(X_test_scaled, y_test)))

Best best_C found: 100
Best best_kernel found: rbf
Best best_gamma found: 0.2782559402207126
Best average score: 0.9961476725521671
Score on training/validation set: 0.9971107544141252
Score on test set: 0.9980750721847931


In [47]:
#Predict test data set.
y_pred = svm.predict(X_test_scaled)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       835
           1       1.00      1.00      1.00       204

    accuracy                           1.00      1039
   macro avg       1.00      1.00      1.00      1039
weighted avg       1.00      1.00      1.00      1039



0.9969502172126334

# Tuning hyperparameters with cross validation split and standardization

In [48]:
# Divide the data into training and test , no validation data

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)

# preprocessing using zero mean and unit variance scaling
scaler = StandardScaler()
scaler.fit(X_train)

X_test_scaled = scaler.transform(X_test)
X_trainval_scaled = scaler.transform( X_trainval)


print("Size of training set:{}".format(X_trainval_scaled.shape[0]))
print("Size of test set:{}".format(X_test_scaled.shape[0]))

Size of training set:3115
Size of test set:1039


In [49]:
best_score = 0

for C in [100, 10, 1.0, 0.1, 0.01, 0.001]:
    for kernel in ['linear', 'rbf']:
        for gamma in 10**np.linspace(-5, 5, 10):
            # Learn the model 
            svm = SVC(kernel=kernel, C=C, gamma = gamma)
            
            # Perform cross validation
            scores = cross_val_score(svm, X_trainval_scaled, y_trainval, cv=5)
        
            # Compute the mean score
            score = scores.mean()
        
        
            # If improvement, store score and parameter
            if score>best_score:
                best_score = score
                best_C = C
                best_kernel= kernel
                best_gamma= gamma

# Build a model on the combine training and valiation data
svm = SVC(kernel= best_kernel, C=best_C, gamma = best_gamma)
svm.fit(X_trainval_scaled, y_trainval)

print("Best best_C found: {}".format(best_C))
print("Best best_kernel found: {}".format(best_kernel))
print("Best best_gamma found: {}".format(best_gamma))
print("Best average score: {}".format(best_score))
print("Score on training/validation set: {}".format(svm.score(X_trainval_scaled, y_trainval)))
print("Score on test set: {}".format(svm.score(X_test_scaled, y_test)))

Best best_C found: 100
Best best_kernel found: linear
Best best_gamma found: 1e-05
Best average score: 0.9974317817014444
Score on training/validation set: 0.9990369181380417
Score on test set: 0.9980750721847931


In [50]:
#Predict test data set.
y_pred = svm.predict(X_test_scaled)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       835
           1       1.00      1.00      1.00       204

    accuracy                           1.00      1039
   macro avg       1.00      1.00      1.00      1039
weighted avg       1.00      1.00      1.00      1039



0.9969502172126334